In [ ]:
# from tqdm import tqdm
import numpy as np
import os
from torchvision import models, transforms
model = models.vgg16(pretrained=True)
import cv2
# from google.colab.patches import cv2_imshow
import torch
from torch import optim, nn
from numpy.ma.core import argmin
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import pairwise_distances
import matplotlib.pyplot as plt
from keras.preprocessing import image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from yellowbrick.cluster import KElbowVisualizer

In [ ]:
class FeatureExtractor(nn.Module):
    def __init__(self, model):
        super(FeatureExtractor, self).__init__()
            # Extract VGG-16 Feature Layers
        self.features = list(model.features)
        self.features = nn.Sequential(*self.features)
            # Extract VGG-16 Average Pooling Layer
        self.pooling = model.avgpool
            # Convert the image into one-dimensional vector
        self.flatten = nn.Flatten()
            # Extract the first part of fully-connected layer from VGG16
        self.fc = model.classifier[0]
  
    def forward(self, x):
        # It will take the input 'x' until it returns the feature vector called 'out'
        out = self.features(x)
        out = self.pooling(out)
        out = self.flatten(out)
        out = self.fc(out) 
        return out 

# Initialize the model
model = models.vgg16(pretrained=True)
new_model = FeatureExtractor(model)

# Change the device to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
new_model = new_model.to(device)

In [ ]:
new_model

In [ ]:
# Transform the image, so it becomes readable with the model
transform = transforms.Compose([
  transforms.ToPILImage(),
  transforms.CenterCrop(512),
  transforms.Resize(448),
  transforms.ToTensor()                              
])

features = []
x=[]
path = '/kaggle/input/h-and-m-personalized-fashion-recommendations/images/'

for root, directories, files in os.walk(path):
  for filename in files:
    
    if filename.endswith((".jpg", ".jpeg", ".png")):
      
      image_path = os.path.join(root, filename)
      x.append(image_path)  
print("Total images:", len(x))

In [ ]:
y=x[0:10000]
for i in y:
    # Set the image path
    #imagePath = os.path.join(path, str(i))
    # Read the file
    img = cv2.imread(i)
    # Transform the image
    img = transform(img)
    # Reshape the image. PyTorch model reads 4-dimensional tensor
    # [batch_size, channels, width, height]
    img = img.reshape(1, 3, 448, 448)
    img = img.to(device)
    # We only extract features, so we don't need gradient
    with torch.no_grad():
      # Extract the feature from the image
      feature = new_model(img)
    # Convert to NumPy Array, Reshape it, and save it to features variable
    features.append(feature.cpu().detach().numpy().reshape(-1))



In [ ]:
len(features)

In [ ]:
# Convert to NumPy Array
features = np.array(features)
print(features.shape)
print(features[0])

In [ ]:
# Initialize the model
min_num_clusters = 15
max_num_clusters = 50
silhouette_scores = []
davies_bouldin_scores = []
# print("Silhouette Scores")

print("k \t Silhouette \t DB")
for num_clusters in range(min_num_clusters, max_num_clusters):
    print()
    model = KMeans(n_clusters=num_clusters)

    # Fit the data into the model
    model.fit(features)

    # Extract the labels
    labels = model.labels_

    # Calculate Silhouette score
    s = metrics.silhouette_score(features, labels, metric='euclidean')
    silhouette_scores.append(s)
    d = metrics.davies_bouldin_score(features, labels)
    davies_bouldin_scores.append(d)
    print(num_clusters, "\t", s, "\t", d)


silhouette_scores = np.array(silhouette_scores)
davies_bouldin_scores = np.array(davies_bouldin_scores)

In [ ]:
model = KMeans()
visualizer = KElbowVisualizer(model, k=(min_num_clusters, max_num_clusters))
visualizer.fit(features)    # Fit the data to the visualizer
visualizer.show()           # Finalize and render the figure

In [ ]:
best_num_clusters = visualizer.elbow_value_
# best_num_clusters = visualizer.elbow_
best_num_clusters_index = best_num_clusters - min_num_clusters  

print("Optimal no. of clusters is", best_num_clusters, "by Elbow Method")


# best_num_clusters_index = np.argmax(silhouette_scores)
# best_num_clusters = min_num_clusters + best_num_clusters_index
# print("Silhouette Scores")
# for i in range(len(silhouette_scores)):
#   print(i, ":", silhouette_scores[i])

# print("Optimal no. of clusters is", best_num_clusters, "with Silhouette score of", silhouette_scores[best_num_clusters_index])


# best_num_clusters_index = np.argmax(davies_bouldin_scores)
# best_num_clusters = min_num_clusters + best_num_clusters_index 
# # print("Silhouette Scores")
# # for i in range(len(silhouette_scores)):
# #   print(i, ":", silhouette_scores[i])

# print("Optimal no. of clusters is", best_num_clusters, "with David Bouldin score of", davies_bouldin_scores[best_num_clusters_index])

In [ ]:
num_clusters = best_num_clusters
model = KMeans(n_clusters=num_clusters)
model.fit(features)
labels = model.labels_
labels = np.array(labels)

In [ ]:
for k in range(5, 10):
    num = 0
    print("\nCluster ", k)
    numImages = len(np.where(labels == k)[0])
    # print(numImages)
    numRows = int(numImages / 10) + 1
    numCols = 10
    fig = plt.figure(k, figsize=(numCols * 5.0  , numRows * 5.0))
    for i in range(labels.shape[0]):
        if(labels[i] == k):
            #imagePath = os.path.join(path + '/', str(images[i]))
            #imagepath=y[i]
            #img = image.load_img(imagePath)
            img = image.load_img(x[i])
            plt.subplot(numRows, numCols, num+1)
            plt.xticks([])
            plt.yticks([])
            plt.imshow(img)
            num += 1
    plt.show()
# print(labels) # [4 3 3 ... 0 0 0]

In [ ]:
test_features = []
test_path = '/kaggle/input/h-and-m-personalized-fashion-recommendations/images/022'
test_images = os.listdir(test_path)
print("Total test images:", len(test_images))

# Iterate each image
for i in test_images:
    # Set the image path
    # ignore non-image data (if any)
    if(str(i)[-4:] != '.jpg'):
        continue
    imagePath = os.path.join(test_path + '/', str(i))
    # Read the file
    img = cv2.imread(imagePath)
    # Transform the image
    img = transform(img)
    # Reshape the image. PyTorch model reads 4-dimensional tensor
    # [batch_size, channels, width, height]
    img = img.reshape(1, 3, 448, 448)
    img = img.to(device)
    # We only extract features, so we don't need gradient
    with torch.no_grad():
      # Extract the feature from the image
      feature = new_model(img)
    # Convert to NumPy Array, Reshape it, and save it to features variable
    test_features.append(feature.cpu().detach().numpy().reshape(-1))

# Convert to NumPy Array
test_features = np.array(test_features)

# Predict the clusters for test data
predicted_labels = model.predict(test_features)
# print(predicted_labels)

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

for i in range(len(predicted_labels)):
    k = predicted_labels[i]
    curr_cluster = np.argwhere(labels == k).flatten()
    dist = euclidean_distances(test_features[i].reshape((1, -1)), features[curr_cluster])
    top_5 = curr_cluster[np.argsort(dist[0])[0:5]]
    numRows = 1
    numCols = 6
    fig = plt.figure(i, figsize=(numCols * 5.0  , numRows * 5.0))
    imagePath = os.path.join(test_path + '/', str(test_images[i]))
    img1 = image.load_img(imagePath)
    plt.subplot(numRows, numCols, 1)
    plt.xticks([])
    plt.yticks([])
    plt.title("Original:")
    plt.imshow(img1)
    num = 1
    for j in range(5):
        imagePath = os.path.join(path + '/', str(y[top_5[j]]))
        img = image.load_img(imagePath)
        plt.subplot(numRows, numCols, num+1)
        plt.xticks([])
        plt.yticks([])
        plt.title("Top: " + str(j+1))
        plt.imshow(img)
        num += 1
    plt.show()